***GENERATED CODE FOR globalchurn PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'order_id', 'transformation_label': 'String Indexer'}], 'feature': 'order_id', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'AG-2013-DC32853-41506', 'max': 'ZA-2015-DD3570146-42306', 'missing': '0', 'distinct': '403'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'order_id'}, {'feature_label': 'order_id', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('order_id')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'order_date', 'transformation_label': 'String Indexer'}], 'feature': 'order_date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/1/2014', 'max': '9/9/2012', 'missing': '0', 'distinct': '335'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'order_date'}, {'feature_label': 'order_date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('order_date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ship_date', 'transformation_label': 'String Indexer'}], 'feature': 'ship_date', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '1/1/2013', 'max': '9/9/2015', 'missing': '0', 'distinct': '370'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ship_date'}, {'feature_label': 'ship_date', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ship_date')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ship_mode', 'transformation_label': 'String Indexer'}], 'feature': 'ship_mode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'First Class', 'max': 'Standard Class', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ship_mode'}, {'feature_label': 'ship_mode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ship_mode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'customer_id', 'transformation_label': 'String Indexer'}], 'feature': 'customer_id', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AA-1037582', 'max': 'YC-2189518', 'missing': '0', 'distinct': '353'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'customer_id'}, {'feature_label': 'customer_id', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('customer_id')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'customer_name', 'transformation_label': 'String Indexer'}], 'feature': 'customer_name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Adam Bellavance', 'max': 'Yoseph Carroll', 'missing': '0', 'distinct': '332'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'customer_name'}, {'feature_label': 'customer_name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('customer_name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'segment', 'transformation_label': 'String Indexer'}], 'feature': 'segment', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Consumer', 'max': 'Home Office', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'segment'}, {'feature_label': 'segment', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('segment')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'city', 'transformation_label': 'String Indexer'}], 'feature': 'city', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Aalst', 'max': 'Águas Lindas de Goiás', 'missing': '0', 'distinct': '288'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'city'}, {'feature_label': 'city', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('city')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'state', 'transformation_label': 'String Indexer'}], 'feature': 'state', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': "'Ajman", 'max': 'Zealand', 'missing': '0', 'distinct': '226'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'state'}, {'feature_label': 'state', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('state')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'country', 'transformation_label': 'String Indexer'}], 'feature': 'country', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Afghanistan', 'max': 'Zambia', 'missing': '0', 'distinct': '84'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'country'}, {'feature_label': 'country', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('country')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'region', 'transformation_label': 'String Indexer'}], 'feature': 'region', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Canada', 'max': 'Western US', 'missing': '0', 'distinct': '23'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'region'}, {'feature_label': 'region', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('region')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'market', 'transformation_label': 'String Indexer'}], 'feature': 'market', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Africa', 'max': 'USCA', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'market'}, {'feature_label': 'market', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('market')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'product_id', 'transformation_label': 'String Indexer'}], 'feature': 'product_id', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'FUR-BO-3621', 'max': 'TEC-PH-6427', 'missing': '0', 'distinct': '426'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'product_id'}, {'feature_label': 'product_id', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('product_id')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'category', 'transformation_label': 'String Indexer'}], 'feature': 'category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Furniture', 'max': 'Technology', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'category'}, {'feature_label': 'category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'sub_category', 'transformation_label': 'String Indexer'}], 'feature': 'sub_category', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Accessories', 'max': 'Tables', 'missing': '0', 'distinct': '17'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'sub_category'}, {'feature_label': 'sub_category', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('sub_category')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'product_name', 'transformation_label': 'String Indexer'}], 'feature': 'product_name', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '"Acco Pressboard Covers with Storage Hooks, 9 1/2"" x 11""', 'max': 'Xerox Cards & Envelopes, Recycled', 'missing': '0', 'distinct': '426'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'product_name'}, {'feature_label': 'product_name', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('product_name')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'sales', 'transformation_label': 'String Indexer'}], 'feature': 'sales', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '100.3', 'stddev': '420.0', 'min': ' 2/Pack"', 'max': '99.84', 'missing': '0', 'distinct': '475'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'sales'}, {'feature_label': 'sales', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('sales')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'quantity', 'transformation_label': 'String Indexer'}], 'feature': 'quantity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '5.86', 'stddev': '2.94', 'min': '1', 'max': '9', 'missing': '0', 'distinct': '18'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'quantity'}, {'feature_label': 'quantity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('quantity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'discount', 'transformation_label': 'String Indexer'}], 'feature': 'discount', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '0.29', 'stddev': '0.77', 'min': '0', 'max': '7', 'missing': '0', 'distinct': '24'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'discount'}, {'feature_label': 'discount', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('discount')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'order_priority', 'transformation_label': 'String Indexer'}], 'feature': 'order_priority', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '1.49', 'stddev': '0.25', 'min': '1.21', 'max': 'Medium', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'order_priority'}, {'feature_label': 'order_priority', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('order_priority')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run globalchurnHooks.ipynb
try:
	#sourcePreExecutionHook()

	orders = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/orders.csv', 'filename': 'orders.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/numtraPlatform/NumtraPlatformV3/uploads/platform/'}")
	#sourcePostExecutionHook(orders)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run globalchurnHooks.ipynb
try:
	#transformationPreExecutionHook()

	globalchurnautofe = TransformationMain.run(orders,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "row_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "29971.03", "stddev": "14890.47", "min": "11", "max": "51268", "missing": "0"}, "updatedLabel": "row_id"}, {"transformationsData": [{"feature_label": "order_id", "transformation_label": "String Indexer"}], "feature": "order_id", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AG-2013-DC32853-41506", "max": "ZA-2015-DD3570146-42306", "missing": "0", "distinct": "403"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "order_id"}, {"transformationsData": [{"feature_label": "order_date", "transformation_label": "String Indexer"}], "feature": "order_date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/1/2014", "max": "9/9/2012", "missing": "0", "distinct": "335"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "order_date"}, {"transformationsData": [{"feature_label": "ship_date", "transformation_label": "String Indexer"}], "feature": "ship_date", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "1/1/2013", "max": "9/9/2015", "missing": "0", "distinct": "370"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ship_date"}, {"transformationsData": [{"feature_label": "ship_mode", "transformation_label": "String Indexer"}], "feature": "ship_mode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "First Class", "max": "Standard Class", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ship_mode"}, {"transformationsData": [{"feature_label": "customer_id", "transformation_label": "String Indexer"}], "feature": "customer_id", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AA-1037582", "max": "YC-2189518", "missing": "0", "distinct": "353"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "customer_id"}, {"transformationsData": [{"feature_label": "customer_name", "transformation_label": "String Indexer"}], "feature": "customer_name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Adam Bellavance", "max": "Yoseph Carroll", "missing": "0", "distinct": "332"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "customer_name"}, {"transformationsData": [{"feature_label": "segment", "transformation_label": "String Indexer"}], "feature": "segment", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Consumer", "max": "Home Office", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "segment"}, {"transformationsData": [{"feature_label": "city", "transformation_label": "String Indexer"}], "feature": "city", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aalst", "max": "\u00c1guas Lindas de Goi\u00e1s", "missing": "0", "distinct": "288"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "city"}, {"transformationsData": [{"feature_label": "state", "transformation_label": "String Indexer"}], "feature": "state", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "'Ajman", "max": "Zealand", "missing": "0", "distinct": "226"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "state"}, {"transformationsData": [{"feature_label": "country", "transformation_label": "String Indexer"}], "feature": "country", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Afghanistan", "max": "Zambia", "missing": "0", "distinct": "84"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "country"}, {"transformationsData": [{"feature_label": "region", "transformation_label": "String Indexer"}], "feature": "region", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Canada", "max": "Western US", "missing": "0", "distinct": "23"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "region"}, {"transformationsData": [{"feature_label": "market", "transformation_label": "String Indexer"}], "feature": "market", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Africa", "max": "USCA", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "market"}, {"transformationsData": [{"feature_label": "product_id", "transformation_label": "String Indexer"}], "feature": "product_id", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "FUR-BO-3621", "max": "TEC-PH-6427", "missing": "0", "distinct": "426"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "product_id"}, {"transformationsData": [{"feature_label": "category", "transformation_label": "String Indexer"}], "feature": "category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Furniture", "max": "Technology", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "category"}, {"transformationsData": [{"feature_label": "sub_category", "transformation_label": "String Indexer"}], "feature": "sub_category", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Accessories", "max": "Tables", "missing": "0", "distinct": "17"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "sub_category"}, {"transformationsData": [{"feature_label": "product_name", "transformation_label": "String Indexer"}], "feature": "product_name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "\"Acco Pressboard Covers with Storage Hooks, 9 1/2\"\" x 11\"\"", "max": "Xerox Cards & Envelopes, Recycled", "missing": "0", "distinct": "426"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "product_name"}, {"transformationsData": [{"feature_label": "sales", "transformation_label": "String Indexer"}], "feature": "sales", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "100.3", "stddev": "420.0", "min": " 2/Pack\"", "max": "99.84", "missing": "0", "distinct": "475"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "sales"}, {"transformationsData": [{"feature_label": "quantity", "transformation_label": "String Indexer"}], "feature": "quantity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "5.86", "stddev": "2.94", "min": "1", "max": "9", "missing": "0", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "quantity"}, {"transformationsData": [{"feature_label": "discount", "transformation_label": "String Indexer"}], "feature": "discount", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "0.29", "stddev": "0.77", "min": "0", "max": "7", "missing": "0", "distinct": "24"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "discount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "profit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "361.43", "stddev": "642.7", "min": "-3009.435", "max": "4946.37", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "profit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "shipping_cost", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "16.77", "stddev": "21.87", "min": "1.06", "max": "431.68", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "shipping_cost"}, {"transformationsData": [{"feature_label": "order_priority", "transformation_label": "String Indexer"}], "feature": "order_priority", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "1.49", "stddev": "0.25", "min": "1.21", "max": "Medium", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "order_priority"}]}))

	#transformationPostExecutionHook(globalchurnautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run globalchurnHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(globalchurnautofe, ["row_id", "profit", "shipping_cost", "order_id_stringindexer", "order_date_stringindexer", "ship_date_stringindexer", "ship_mode_stringindexer", "customer_id_stringindexer", "customer_name_stringindexer", "segment_stringindexer", "city_stringindexer", "state_stringindexer", "country_stringindexer", "market_stringindexer", "product_id_stringindexer", "category_stringindexer", "sub_category_stringindexer", "product_name_stringindexer", "sales_stringindexer", "quantity_stringindexer", "discount_stringindexer", "order_priority_stringindexer"], "region_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

